In [1]:
import cv2
import sys
import random
import os
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import math_ops
import time
from nltk.tokenize import RegexpTokenizer
import operator


In [2]:
DATA_PATH = "/home/will/cvdata/"
OUTPUT_PATH = "/home/will/cvdata/imgs-halfhalf/"
CROPPED_PATH = "/home/will/sdb1/imgs-halfhalf10/"
STORE_PATH = "/home/will/sdb1/"

In [3]:
def png2jpg(inf):
    '''convert pngs in path to jpg'''
    if os.path.isfile(inf) and inf[-3:] is "png":
        # if img is file
        img = cv2.imread(inf)
        cv2.imwrite(OUTPUT_PATH + inf.split('/')[-1][:-3] + 'jpg', img)
    elif os.path.isdir(inf):
        pngs = glob(inf+'/*.png')
        for j in pngs:
            print("processing " + j)
            img = cv2.imread(j)
            cv2.imwrite(OUTPUT_PATH + j.split('/')[-1][:-3] + 'jpg', img)

In [4]:
def crop10(img, h, w):
    '''crop 10 random sub images from original with (h, w) size'''
    sub_imgs = []
    num_imgs = 10
    oh, ow, _ = img.shape
    if w > ow or h > oh:
        return sub_imgs
    for i in range(num_imgs):
        # get the size of the sub-image
        sh = random.randint(1, oh-h)
        sw = random.randint(1, ow-w)
        temp = img[sh:sh+h, sw:sw+w]
        sub_imgs.append(temp)
        
    return sub_imgs

In [5]:
def print_progress(iteration, total, prefix='PROG', suffix='',
                   decimals=1, length=25, fill='>'):
    percent = ("{0:." +
               str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    sys.stdout.write('\r[%s] %s: |%s| %s%% %s' % (str(datetime.now().strftime("%y%m%d-%H%M%S")),
                                                prefix, bar, percent, suffix))
    # Print New Line on Complete
    if iteration == total:
        print("")

In [6]:
net_data = np.load(open(DATA_PATH + "bvlc_alexnet.npy", "rb"), encoding="latin1").item()

In [7]:
train_x = np.zeros((1, 227,227,3)).astype(np.float32)
train_y = np.zeros((1, 1000))
xdim = train_x.shape[1:]
ydim = train_y.shape[1]

In [8]:
def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w,  padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    assert c_i%group==0
    assert c_o%group==0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    
    
    if group==1:
        conv = convolve(input, kernel)
    else:
        #tf.split(value, num_or_size_splits, axis) -- new version
        ## tf.split(axis, num_or_size_splits, value) -- old version
        
        #input_groups = tf.split(3, group, input)
        #kernel_groups = tf.split(3, group, kernel)
        
        input_groups = tf.split(input, group, 3)
        kernel_groups = tf.split(kernel, group, 3)
        
        output_groups = [convolve(i, k) for i,k in zip(input_groups, kernel_groups)]
        conv = tf.concat( output_groups, 3)
    return  tf.reshape(tf.nn.bias_add(conv, biases), [-1]+conv.get_shape().as_list()[1:])



x = tf.placeholder(tf.float32, (None,) + xdim)

In [9]:
k_h = 11; k_w = 11; c_o = 96; s_h = 4; s_w = 4
conv1W = tf.Variable(net_data["conv1"][0])
conv1b = tf.Variable(net_data["conv1"][1])
conv1_in = conv(x, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
conv1 = tf.nn.relu(conv1_in)

#lrn1
#lrn(2, 2e-05, 0.75, name='norm1')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn1 = tf.nn.local_response_normalization(conv1,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool1
#max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)


#conv2
#conv(5, 5, 256, 1, 1, group=2, name='conv2')
k_h = 5; k_w = 5; c_o = 256; s_h = 1; s_w = 1; group = 2
conv2W = tf.Variable(net_data["conv2"][0])
conv2b = tf.Variable(net_data["conv2"][1])
conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv2 = tf.nn.relu(conv2_in)


#lrn2
#lrn(2, 2e-05, 0.75, name='norm2')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn2 = tf.nn.local_response_normalization(conv2,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool2
#max_pool(3, 3, 2, 2, padding='VALID', name='pool2')                                                  
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#conv3
#conv(3, 3, 384, 1, 1, name='conv3')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 1
conv3W = tf.Variable(net_data["conv3"][0])
conv3b = tf.Variable(net_data["conv3"][1])
conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv3 = tf.nn.relu(conv3_in)

#conv4
#conv(3, 3, 384, 1, 1, group=2, name='conv4')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 2
conv4W = tf.Variable(net_data["conv4"][0])
conv4b = tf.Variable(net_data["conv4"][1])
conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv4 = tf.nn.relu(conv4_in)


#conv5
#conv(3, 3, 256, 1, 1, group=2, name='conv5')
k_h = 3; k_w = 3; c_o = 256; s_h = 1; s_w = 1; group = 2
conv5W = tf.Variable(net_data["conv5"][0])
conv5b = tf.Variable(net_data["conv5"][1])
conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)

conv5 = tf.nn.relu(conv5_in)

#maxpool5
#max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#fc6
#fc(4096, name='fc6')
fc6W = tf.Variable(net_data["fc6"][0])
fc6b = tf.Variable(net_data["fc6"][1])

#fc6 before relu
fc6_before_relu = nn_ops.bias_add( math_ops.matmul(tf.reshape(maxpool5, [-1, int(np.prod(maxpool5.get_shape()[1:]))]), fc6W)   ,fc6b)

fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [-1, int(np.prod(maxpool5.get_shape()[1:]))]), fc6W, fc6b)

#fc7
#fc(4096, name='fc7')
fc7W = tf.Variable(net_data["fc7"][0])
fc7b = tf.Variable(net_data["fc7"][1])
fc7 = tf.nn.relu_layer(fc6, fc7W, fc7b)

fc7_xw_plus_b = nn_ops.bias_add(math_ops.matmul(fc6, fc7W), fc7b)


fc7_after_relu = nn_ops.relu(fc7_xw_plus_b)


#fc8
#fc(1000, relu=False, name='fc8')
fc8W = tf.Variable(net_data["fc8"][0])
fc8b = tf.Variable(net_data["fc8"][1])
fc8 = tf.nn.xw_plus_b(fc7, fc8W, fc8b)


#prob
#softmax(name='prob'))
prob = tf.nn.softmax(fc8)

#init = tf.initialize_all_variables()
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

t = time.time()

In [10]:
import PIL
from PIL import Image
import skimage

In [58]:
# Crop images
# h = 227
# w = 227
# count = 0
# total = len(os.listdir(OUTPUT_PATH))
# for item in os.listdir(OUTPUT_PATH):
#     count += 1
#     img_path = OUTPUT_PATH + item
#     img = cv2.imread(img_path)
#     imgs10 = crop10(img, h, w)
#     print(count)
#     if len(imgs10) is not 10:
#         continue
#     for i in range(10):
#         cv2.imwrite(CROPPED_PATH + item[:-4]+ "-part" + str(i) + ".jpg", imgs10[i])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077


KeyboardInterrupt: 

In [11]:
datasize = 500
image_list = []
raw_image_list = []
count = 0
for item in sorted(os.listdir(CROPPED_PATH)):
    count += 1
    if datasize < count:
        break
    im = cv2.imread(CROPPED_PATH + item)
    raw_image_list.append(im)
    im = (im).astype(np.float32)
    im = im - np.mean(im)
    im[:, :, 0], im[:, :, 2] = im[:, :, 2], im[:, :, 0]
    image_list.append(im)

In [12]:
print(len(image_list))

500


In [13]:
im_fc7 = sess.run(fc7_xw_plus_b, feed_dict = {x:image_list})
im_fc7 = np.array(im_fc7)

im_fc6 = sess.run(fc6_before_relu, feed_dict = {x:image_list})
im_fc6 = np.array(im_fc6)

im_conv5 = sess.run(conv5_in, feed_dict = {x:image_list})
im_conv5 = np.array(im_conv5)

In [14]:
dataset = 'patents'

np.save(STORE_PATH + dataset +'_raw_image_list.npy', raw_image_list)
raw_image_list = np.array(raw_image_list)
print(raw_image_list.shape)

np.save(STORE_PATH + dataset + '_image_list.npy', image_list)
image_list = np.array(image_list)
print(image_list.shape)

(500, 227, 227, 3)
(500, 227, 227, 3)


In [15]:
np.save(STORE_PATH + dataset + '_fc7.npy', im_fc7)
print(im_fc7.shape)

np.save(STORE_PATH + dataset + '_fc6.npy', im_fc6)
print(im_fc6.shape)

np.save(STORE_PATH + dataset + '_conv5.npy', im_conv5)
print(im_conv5.shape)

(500, 4096)
(500, 4096)
(500, 13, 13, 256)


In [16]:
'''get threshold using 90% percentile'''
percentile_fc7 = np.percentile(im_fc7, 90, axis = 0)
percentile_fc6 = np.percentile(im_fc6, 90, axis = 0)
percentile_conv5 = np.percentile(im_conv5, 90, axis = 0)

In [17]:
np.save(STORE_PATH +  dataset +'_percentile_fc7.npy', percentile_fc7)
np.save(STORE_PATH +  dataset +'_percentile_fc6.npy', percentile_fc6)
np.save(STORE_PATH +  dataset +'_percentile_conv5.npy', percentile_conv5)

percentile_fc7 = np.array(percentile_fc7)
print(percentile_fc7.shape)
percentile_fc6 = np.array(percentile_fc6)
print(percentile_fc6.shape)
percentile_conv5 = np.array(percentile_conv5)
print(percentile_conv5.shape)

(4096,)
(4096,)
(13, 13, 256)


In [18]:
# Topics count (7, 20, 50, 100)
NUM_TOPIC = datasize / 10
# Features for Topics
NUM_FEATURE = 4096
# Amount of images being classified
NUM_IMAGE = datasize

In [19]:
X = im_fc7

In [20]:
# Get data statistics, as a sanity check
print("Daxta shape: ", X.shape)
print("Number of 0s: ", np.sum(X == 1))
print("Number of 1s: ", np.sum(X == 0))
print("Anomailes: ",np.sum([X < 0]))


('Daxta shape: ', (500, 4096))
('Number of 0s: ', 0)
('Number of 1s: ', 0)
('Anomailes: ', 1755369)


In [21]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gensim import corpora, models

In [22]:
binary_vector_fc7 = np.greater(im_fc7, percentile_fc7).astype(int)

In [23]:
print("Data shape: ", binary_vector_fc7.shape)
print("Number of 1s: ", np.sum(binary_vector_fc7 == 1))
print("Number of 0s: ", np.sum(binary_vector_fc7 == 0))
print("Anomailes: ",np.sum([binary_vector_fc7 < 0]))


('Data shape: ', (500, 4096))
('Number of 1s: ', 201127)
('Number of 0s: ', 1846873)
('Anomailes: ', 0)


In [24]:
X = binary_vector_fc7
DATA_PATH = "/home/will/cvdata/workspace/data2/"

In [25]:
# Prepare for corpus
corpus = [[(j, X[i, j]) for j in range(NUM_FEATURE) if X[i, j]==1] for i in range(NUM_IMAGE)]
corpora.MmCorpus.serialize('data/corpus.mm', corpus)

In [26]:
curr_time = str(int(time.time()))
model_name = str(NUM_TOPIC) + '-topics.model'
topics_file_name = 'topic_model_features_' + str(NUM_TOPIC) +'_topics_'+ curr_time + '.npy'
topics_per_image_file_name = 'topics_per_image-' + str(NUM_TOPIC) +'_topics_'+ curr_time + '.npy'
print("Loading model: " + model_name)

Loading model: 50-topics.model


In [27]:
# load trained data
lda = models.ldamodel.LdaModel.load(DATA_PATH + model_name)
print(lda)

LdaModel(num_terms=4096, num_topics=50, decay=0.5, chunksize=2000)


In [28]:
# Process the raw data
doc = []
bow = []
for i in range(NUM_IMAGE):
    doc_list = []
    bow_list = []
    for j in range(X.shape[1]): # 4096
        doc_list.append((j, X[i,j]))
        bow_list.append(X[i,j])
    doc.append(doc_list)
    bow.append(bow_list)
    

In [29]:
# Raw output for dist
topic_distribution = lda.print_topics(num_words=NUM_FEATURE)


In [30]:
'''Save distribution of features for every topic'''

# Extract feature ids per topic from raw output
topics = np.zeros([NUM_TOPIC, NUM_FEATURE])
tokenizer = RegexpTokenizer(r'\w+')

for topic_ids in topic_distribution:
    topic_id = topic_ids[0]
    all_features = topic_ids[1]

    feature_values = tokenizer.tokenize(all_features)     # Tokenize the string to keep all numbers
    feature_values = list(map(int, feature_values))       # Convert values of list to int
    feature_values = np.asarray(feature_values[2::3]) * np.asarray(feature_values[1::3])   # Remove all features with 0 weightage in topic
    
    topics[topic_id] = feature_values
        
# Save feature values in npy file
np.save(DATA_PATH + topics_file_name, topics)

#topics is a numpy array with one row representing one topic. The columns contain the indices of the features belonging to that topic, in decreasing order of how strongly linked a feature is to that topic. The vectors have been padded with 0s in the end.

In [31]:
'''Save distribution of topics, for every image'''

# Save top k topics per image to file
k = 5

topics_per_image = lda[doc]
topics_per_image_matrix = np.zeros([len(topics_per_image),k,2])
print ("No of documents: ", len(topics_per_image))

i = 0
topic_dict = {}
for image_topics in topics_per_image:
    image_topics.sort(key=operator.itemgetter(1), reverse=True)
    print ("Image ",i, " Topics: ", image_topics[0][0])
    if image_topics[0][0] not in topic_dict:
        topic_dict[image_topics[0][0]] = []
    topic_dict[image_topics[0][0]].append(i)
    for j in range(min(k,len(image_topics))):
        topics_per_image_matrix[i][j][0] = image_topics[j][0]    # Store topic id
        topics_per_image_matrix[i][j][1] = image_topics[j][1]    # Store probability of document having that topic
    i = i+1
    
# Save feature values in npy file
np.save(DATA_PATH + topics_per_image_file_name, topics_per_image_matrix)

('No of documents: ', 500)
('Image ', 0, ' Topics: ', 34)
('Image ', 1, ' Topics: ', 37)
('Image ', 2, ' Topics: ', 20)
('Image ', 3, ' Topics: ', 9)
('Image ', 4, ' Topics: ', 37)
('Image ', 5, ' Topics: ', 2)
('Image ', 6, ' Topics: ', 37)
('Image ', 7, ' Topics: ', 37)
('Image ', 8, ' Topics: ', 2)
('Image ', 9, ' Topics: ', 43)
('Image ', 10, ' Topics: ', 35)
('Image ', 11, ' Topics: ', 37)
('Image ', 12, ' Topics: ', 37)
('Image ', 13, ' Topics: ', 49)
('Image ', 14, ' Topics: ', 37)
('Image ', 15, ' Topics: ', 29)
('Image ', 16, ' Topics: ', 30)
('Image ', 17, ' Topics: ', 28)
('Image ', 18, ' Topics: ', 6)
('Image ', 19, ' Topics: ', 37)
('Image ', 20, ' Topics: ', 34)
('Image ', 21, ' Topics: ', 37)
('Image ', 22, ' Topics: ', 3)
('Image ', 23, ' Topics: ', 34)
('Image ', 24, ' Topics: ', 2)
('Image ', 25, ' Topics: ', 3)
('Image ', 26, ' Topics: ', 48)
('Image ', 27, ' Topics: ', 35)
('Image ', 28, ' Topics: ', 34)
('Image ', 29, ' Topics: ', 25)
('Image ', 30, ' Topics: ', 42

('Image ', 267, ' Topics: ', 37)
('Image ', 268, ' Topics: ', 34)
('Image ', 269, ' Topics: ', 31)
('Image ', 270, ' Topics: ', 34)
('Image ', 271, ' Topics: ', 34)
('Image ', 272, ' Topics: ', 34)
('Image ', 273, ' Topics: ', 37)
('Image ', 274, ' Topics: ', 41)
('Image ', 275, ' Topics: ', 34)
('Image ', 276, ' Topics: ', 34)
('Image ', 277, ' Topics: ', 34)
('Image ', 278, ' Topics: ', 34)
('Image ', 279, ' Topics: ', 34)
('Image ', 280, ' Topics: ', 11)
('Image ', 281, ' Topics: ', 8)
('Image ', 282, ' Topics: ', 29)
('Image ', 283, ' Topics: ', 29)
('Image ', 284, ' Topics: ', 29)
('Image ', 285, ' Topics: ', 29)
('Image ', 286, ' Topics: ', 31)
('Image ', 287, ' Topics: ', 29)
('Image ', 288, ' Topics: ', 29)
('Image ', 289, ' Topics: ', 35)
('Image ', 290, ' Topics: ', 29)
('Image ', 291, ' Topics: ', 20)
('Image ', 292, ' Topics: ', 29)
('Image ', 293, ' Topics: ', 29)
('Image ', 294, ' Topics: ', 29)
('Image ', 295, ' Topics: ', 46)
('Image ', 296, ' Topics: ', 29)
('Image ', 

In [32]:
for x in topic_dict:
    print (str(x) + ': ' + str(len(topic_dict[x])))
#     print (topic_dict[x])

0: 20
1: 5
2: 21
3: 15
4: 2
6: 6
7: 3
8: 19
9: 1
11: 5
16: 1
18: 7
20: 17
22: 4
23: 2
25: 7
27: 1
28: 2
29: 127
30: 2
31: 11
32: 1
33: 1
34: 100
35: 32
36: 4
37: 54
38: 2
41: 4
42: 9
43: 4
44: 1
45: 1
46: 3
47: 1
48: 1
49: 4
